In [ ]:
import sys, os
cwd=os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, cwd)
import utils.s3_utils as s3
from utils.xtrain_processing import training_proc
import pandas as pd
import shapefile as shp
import geopandas as gpd
import re
import matplotlib.pyplot as plt
import numpy as np
import folium
import utils.processing as pr
import os
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from config.paths import cov_dict
import config.paths as path
from geocube.api.core import make_geocube

from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_columns', None)

## This notebook processes data into .tif files for further processing in QGIS into .asc images for usage in Maxent. 

In [ ]:
region_code_list = [0,1,2]
modelpath = path.model_path

write_path = f'../../data/{modelpath}/maxent-covariates/processed-cov-tif/'

In [ ]:
env_db_path = '../../data/raw-data/landcover-EEA/eea_r_3035_100_m_ecosystem-types-terrestrial-c_p_2012_v03_r01/mapping-dict-eea_r_3035_100_m_etm-terrestrial-c_2012_v3-1_r00.tif.vat.csv'
env_db = pd.read_csv(env_db_path).rename({'Value,N,10,0':'env_code','EUNIS_L2,C,254':'env_type'},axis=1)
env_map = dict(zip(env_db.env_code,env_db.env_type))
env_map

In [ ]:
covariate_list = list(cov_dict.keys())[1:]
covariate_list

In [ ]:
#read in processed country predictor data
df_list = []
for region_code in region_code_list:
    print(f'Ingesting region {region_code} covariate data...')
    read_filename = f'../../data/{modelpath}/processed-predictor-parquets/clustered/{region_code}-predictors.parquet'
    pred_df = pd.read_parquet(read_filename)
    pred_df = gpd.GeoDataFrame(
        pred_df, geometry=gpd.points_from_xy(pred_df['lon_env'], pred_df['lat_env']))

    pred_df = pred_df.drop_duplicates().reset_index(drop=True)
    pred_df['landcover'] = pred_df['landcover'].astype('int16')
    pred_df['elev'] = pred_df['elev'].astype('int16')

    pred_df.columns = map(str.lower, pred_df.columns)
    pred_df = pred_df.rename({
                    'tg-grp-mean-days-above-5degc-monthly-ratio':'tg-grp-mean-days-above-5degC-monthly-ratio', 
                    }, axis=1)
    pred_df.info()

    for covariate in covariate_list:
        print(covariate)
        
        A = pred_df[['geometry','lon_env','lat_env',covariate]]
        res=.01

        out_grd = make_geocube(
            vector_data=A,
            measurements=[covariate],
            resolution=(-res, res)
        )
        out_grd = out_grd.fillna(-9999)
        print(f'writing {covariate} to file')
        geotif_file = write_path + f'region-{region_code}/{covariate}.tif'
        out_grd[covariate].rio.to_raster(geotif_file, compress="LZW")

### Writing as shapefile... takes forever (not recommended)


In [ ]:
for covariate in covariate_list:
    pred_df[[ 'geometry','lon_env','lat_env',covariate]].reset_index(drop=True).to_file(write_path + f'{covariate}.shp')